In [1]:
# Imports

import sys
import os
import json

import numpy as np
import skimage
import tensorflow as tf

In [2]:
# To make sure if we are using the correct virtual environment

sys.executable

'c:\\college\\major project\\project\\venv\\scripts\\python.exe'

In [3]:
PATH_PREFIX = 'C:\College\Major Project\PROJECT'

DATASET_PATH = f'{PATH_PREFIX}\dataset1'
LOGS_PATH = f'{PATH_PREFIX}\Mask_RCNN\logs'

# Root directory for Mask RCNN
ROOT_DIR = f'{PATH_PREFIX}\Mask_RCNN'

WEIGHTS_PATH = f'{ROOT_DIR}\mask_rcnn_coco.h5'

print(f'DATASET PATH : {DATASET_PATH}')
print(f'LOGS PATH : {LOGS_PATH}')

DATASET PATH : C:\College\Major Project\PROJECT\dataset1
LOGS PATH : C:\College\Major Project\PROJECT\Mask_RCNN\logs


In [4]:
sys.path.append(ROOT_DIR)

# MASCK RCNN IMPORTS

from mrcnn.config import Config
from mrcnn import model as modellib, utils

Using TensorFlow backend.


In [6]:
class FoodConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "food"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # Background + samosa + pizza + cupcake + omelette

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    EPOCHS = 50

In [7]:
config = FoodConfig()

In [8]:
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
EPOCHS                         50
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE     

In [9]:
with tf.device("/dml:0"):
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=LOGS_PATH)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [10]:
model.load_weights(
    WEIGHTS_PATH,
    by_name=True,
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
)

In [14]:
class FoodDataset(utils.Dataset):

    def load_food(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes.
        self.add_class("food", 1, "samosa")
        self.add_class("food", 2, "pizza")
        self.add_class("food", 3, "cupcake")
        self.add_class("food", 4, "omelette")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance.
            # These are stores in the shape_attributes (see json format above).
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]
                
            if type(a['regions']) is dict:
                region_attributes = [r['region_attributes'] for r in a['regions'].values()]
            else:
                region_attributes = [r['region_attributes'] for r in a['regions']]

            num_ids=[]
            for item in region_attributes:
                try:
                    if item['food'] == 'samosa':
                        num_ids.append(1)
                    elif item['food'] =='pizza':
                        num_ids.append(2)
                    elif item['food'] == 'cupcake':
                        num_ids.append(3)
                    elif item['food'] == 'omelette':
                        num_ids.append(4)
                except:
                    pass

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "food",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width,
                height=height,
                polygons=polygons,
                num_ids=num_ids
            )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]

        # If not a food dataset image, delegate to parent class.
        if image_info["source"] != "food":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]

        # Empty mask
        mask = np.zeros(
            [
                info["height"],
                info["width"],
                len(info["polygons"])
            ],
            dtype=np.uint8
        )

        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            x, y = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[x, y, i] = 1
    
        class_ids = image_info['num_ids']
        class_ids = np.reshape(class_ids, [mask.shape[-1]])

        # Return mask, and array of class IDs of each instance.
        return mask.astype(np.bool), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "food":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [12]:
# Training dataset.
dataset_train = FoodDataset()
dataset_train.load_food(DATASET_PATH, "train")
dataset_train.prepare()

In [13]:
# Validation dataset
dataset_val = FoodDataset()
dataset_val.load_food(DATASET_PATH, "val")
dataset_val.prepare()

In [15]:
# model.get_summary()

In [16]:
'''
def train_model():
    config = food.FoodConfig()
    
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=LOGS_PATH)
    
    model.load_weights(
        WEIGHTS_PATH,
        by_name=True,
        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
    )
    
    # Training dataset.
    dataset_train = food.FoodDataset()
    dataset_train.load_food(DATASET_PATH, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = food.FoodDataset()
    dataset_val.load_food(DATASET_PATH, "val")
    dataset_val.prepare()
    
    model.train(
        dataset_train,
        dataset_val,
        learning_rate=config.LEARNING_RATE,
        epochs=5,
        layers='heads'
    )
''';

In [15]:
history = model.train(
    dataset_train,
    dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=config.EPOCHS,
    layers='heads'
)


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\College\Major Project\PROJECT\Mask_RCNN\logs\food20220703T1204\mask_rcnn_food_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistri

c:\college\major project\project\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\college\major project\project\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\college\major project\project\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






Epoch 1/50
50/50 [==============================] - 133s 3s/step - loss: 2.3121 - rpn_class_loss: 0.0122 - rpn_bbox_loss: 0.3157 - mrcnn_class_loss: 0.2637 - mrcnn_bbox_loss: 0.8769 - mrcnn_mask_loss: 0.8436 - val_loss: 1.4600 - val_rpn_class_loss: 0.0064 - val_rpn_bbox_loss: 0.2570 - val_mrcnn_class_loss: 0.1305 - val_mrcnn_bbox_loss: 0.5572 - val_mrcnn_mask_loss: 0.5090

Epoch 2/50
50/50 [==============================] - 128s 3s/step - loss: 1.2233 - rpn_class_loss: 0.0079 - rpn_bbox_loss: 0.2627 - mrcnn_class_loss: 0.1040 - mrcnn_bbox_loss: 0.4405 - mrcnn_mask_loss: 0.4082 - val_loss: 1.0944 - val_rpn_class_loss: 0.0068 - val_rpn_bbox_loss: 0.2328 - val_mrcnn_class_loss: 0.0920 - val_mrcnn_bbox_loss: 0.4704 - val_mrcnn_mask_loss: 0.2924
Epoch 3/50
50/50 [==============================] - 135s 3s/step - loss: 0.9862 - rpn_class_loss: 0.0121 - rpn_bbox_loss: 0.2388 - mrcnn_class_loss: 0.0903 - mrcnn_bbox_loss: 0.4181 - mrcnn_mask_loss: 0.2269 - val_loss: 0.9683 - val_rpn_class_lo

KeyboardInterrupt: 

In [21]:
model.log_dir

'C:\\College\\Major Project\\PROJECT\\Mask_RCNN\\logs\\food20220703T1150'

In [22]:
history.epoch

[0, 1]

In [23]:
history

In [24]:
history.history['loss']

[1.7173338571190835, 0.7917367114126682]

In [25]:
history.history['val_loss']

[0.9772077095508576, 0.8524958974123001]